In [3]:
from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data' 
sub1 = str(wd) + '/subtask1.parquet'
print(sub1)

/home/matijak/Documents/programiranje/projects/semeval/merged_data/subtask1.parquet


In [4]:
import pandas as pd
df = pd.read_parquet(sub1)
df.head()

,lang,art_name,entity,start,end,class1,classes2,text
0,BG,BG_670.txt,Запад,152,156,Antagonist,"[Conspirator, Instigator, Foreign Adversary]",Опитът на колективния Запад да „обезкърви Руси...
1,BG,BG_670.txt,САЩ,530,532,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
2,BG,BG_670.txt,НАТО,535,538,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
3,BG,BG_670.txt,Украйна,578,584,Antagonist,[Foreign Adversary],Опитът на колективния Запад да „обезкърви Руси...
4,BG,BG_670.txt,украински войници,633,649,Innocent,[Victim],Опитът на колективния Запад да „обезкърви Руси...


In [5]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(2)
    if row['class1'] == 'Protagonist':
        return int(1)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)
df.loc[448]

lang                                                       EN
art_name                                     EN_UA_103861.txt
entity                                                Chinese
start                                                     791
end                                                       797
class1                                             Antagonist
classes2                                                [Spy]
text        The World Needs Peacemaker Trump Again \n\n by...
label                                                       0
input       The World Needs Peacemaker Trump Again  by Jef...
Name: 448, dtype: object

In [6]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)
    
    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]
    
    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
df['new_start_end'] = df.apply(adjust_start_end,axis=1)
print(df.loc[448])

lang                                                            EN
art_name                                          EN_UA_103861.txt
entity                                                     Chinese
start                                                          791
end                                                            797
class1                                                  Antagonist
classes2                                                     [Spy]
text             The World Needs Peacemaker Trump Again \n\n by...
label                                                            0
input            The World Needs Peacemaker Trump Again  by Jef...
new_start_end                                           (785, 792)
Name: 448, dtype: object


In [7]:
import spacy

# Load the pre-trained models for each language
nlp_en = spacy.load('en_core_web_sm')  # English model
nlp_bg = spacy.load('xx_ent_wiki_sm')  # Bulgarian model
nlp_hi = spacy.load('xx_ent_wiki_sm')  # Hindi model
nlp_pt = spacy.load('xx_ent_wiki_sm')  # Portuguese model

nlp_en.add_pipe('sentencizer')
nlp_bg.add_pipe('sentencizer')
nlp_hi.add_pipe('sentencizer')
nlp_pt.add_pipe('sentencizer')


In [8]:
#ovo se vrti minutu i pol da se ne cudite
def sentencizer(row):
    sentences = 0
    if row['lang'] == 'EN':
        doc = nlp_en(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    if row['lang'] == 'HI':
        doc = nlp_hi(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    if row['lang'] == 'BG':
        doc = nlp_bg(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    if row['lang'] == 'PT':
        doc = nlp_pt(row['input'])
        sentences = [sentence.text for sentence in doc.sents]
    return sentences
df['sentences'] = df.apply(sentencizer,axis=1)
df.loc[0]

lang                                                            BG
art_name                                                BG_670.txt
entity                                                       Запад
start                                                          152
end                                                            156
class1                                                  Antagonist
classes2              [Conspirator, Instigator, Foreign Adversary]
text             Опитът на колективния Запад да „обезкърви Руси...
label                                                            0
input            Опитът на колективния Запад да „обезкърви Руси...
new_start_end                                           (151, 156)
sentences        [Опитът на колективния Запад да „обезкърви Рус...
Name: 0, dtype: object

In [9]:
def sent_id(row):
    sents = row['sentences']
    inp = row['input']
    start = row['new_start_end'][0]
    ind = 0
    ind_s = -1
    for s in range(len(sents)):
        for i in range(len(sents[s])):
            ind+=1
            if ind == start:
                ind_s = s
                break
        if ind_s > -1:
            break


    if ind_s >= len(sents):
        print("WTF",ind_s,len(sents))
    return ind_s

df['new_ind'] = df.apply(sent_id,axis=1)
print(df.loc[448])
nid = df['new_ind'].loc[448]
sent = df['sentences'].loc[448]
print(sent[nid])

lang                                                            EN
art_name                                          EN_UA_103861.txt
entity                                                     Chinese
start                                                          791
end                                                            797
class1                                                  Antagonist
classes2                                                     [Spy]
text             The World Needs Peacemaker Trump Again \n\n by...
label                                                            0
input            The World Needs Peacemaker Trump Again  by Jef...
new_start_end                                           (785, 792)
sentences        [The World Needs Peacemaker Trump Again  by Je...
new_ind                                                          5
Name: 448, dtype: object
TRUTH LIVES on at https://sgtreport.tv/ There has been an astounding 6,300% increase in the number of Ch

In [10]:
def sent_input(row):
    nind = row['new_ind']
    sents = row['sentences']
    inp = ""
    if nind - 1 >= 0:
        inp+=sents[nind-1]
    inp+=sents[nind]
    if nind + 1 < len(sents):
        inp+=sents[nind+1]
    return inp
def sent_input_short(row):
    nind = int(row['new_ind'])
    sents = row['sentences']
    inp = ""
    if nind >= len(sents) :
        print(nind,len(sents))
    inp+=sents[nind]
    return inp
df['context'] = df.apply(sent_input,axis=1)
df['no_context'] = df.apply(sent_input_short,axis=1)

print(df['context'].loc[448],
    df['no_context'].loc[448])

If this rate continues, this year’s total will easily top the 24,125 Chinese nationals who illegally entered the country last year.TRUTH LIVES on at https://sgtreport.tv/ There has been an astounding 6,300% increase in the number of Chinese nationals illegally entering the country in the last few years.Since China is a communist nation, these individuals are not freely traveling to the United States for a “better life.” TRUTH LIVES on at https://sgtreport.tv/ There has been an astounding 6,300% increase in the number of Chinese nationals illegally entering the country in the last few years.


In [11]:
from transformers import pipeline

# Load a pre-trained model for zero-shot classification (BART)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",device = 0)

In [12]:
df_en = df[df['lang']=='EN']
df_en.reset_index(drop=True)

,lang,art_name,entity,start,end,class1,classes2,text,label,input,new_start_end,sentences,new_ind,context,no_context
0,EN,EN_UA_103861.txt,Chinese,791,797,Antagonist,[Spy],The World Needs Peacemaker Trump Again \n\n by...,0,The World Needs Peacemaker Trump Again by Jef...,"(785, 792)",[The World Needs Peacemaker Trump Again by Je...,5,"If this rate continues, this year’s total will...",TRUTH LIVES on at https://sgtreport.tv/ There ...
1,EN,EN_UA_103861.txt,China,1516,1520,Antagonist,[Instigator],The World Needs Peacemaker Trump Again \n\n by...,0,The World Needs Peacemaker Trump Again by Jef...,"(1508, 1513)",[The World Needs Peacemaker Trump Again by Je...,11,"In addition, Chinese investors are buying Amer...",China is constantly threatening Taiwan and its...
2,EN,EN_UA_103861.txt,Hamas,2121,2125,Antagonist,[Terrorist],The World Needs Peacemaker Trump Again \n\n by...,0,The World Needs Peacemaker Trump Again by Jef...,"(2111, 2116)",[The World Needs Peacemaker Trump Again by Je...,16,There is no end in sight to the war as no peac...,"On October 7, Israel was invaded by Hamas resu..."
3,EN,EN_UA_103861.txt,Donald Trump,4909,4920,Protagonist,"[Peacemaker, Guardian]",The World Needs Peacemaker Trump Again \n\n by...,1,The World Needs Peacemaker Trump Again by Jef...,"(4891, 4903)",[The World Needs Peacemaker Trump Again by Je...,39,As the world deals with multiple international...,Read More @ TheLibertyDaily.com
4,EN,EN_UA_021270.txt,Yermak,667,672,Antagonist,[Incompetent],"Ukraine's Fate Will Be Decided In Coming Year,...",0,"Ukraine's Fate Will Be Decided In Coming Year,...","(662, 668)",[Ukraine's Fate Will Be Decided In Coming Year...,5,He issued the words while appealing for more u...,Yermak sought to assure the audience that Zele...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,EN,EN_UA_027787.txt,the United States,105,121,Antagonist,[Instigator],Russia Says US Is Responsible for Deadly Ukrai...,0,Russia Says US Is Responsible for Deadly Ukrai...,"(103, 120)",[Russia Says US Is Responsible for Deadly Ukra...,0,Russia Says US Is Responsible for Deadly Ukrai...,Russia Says US Is Responsible for Deadly Ukrai...
410,EN,EN_UA_027787.txt,Kremlin,1864,1870,Protagonist,[Guardian],Russia Says US Is Responsible for Deadly Ukrai...,1,Russia Says US Is Responsible for Deadly Ukrai...,"(1851, 1858)",[Russia Says US Is Responsible for Deadly Ukra...,13,"Russia will respond to Sunday’s attack, the De...",The Kremlin said President Vladimir Putin had ...
411,EN,EN_UA_027787.txt,Dmitry Medvedev,1970,1984,Protagonist,[Guardian],Russia Says US Is Responsible for Deadly Ukrai...,1,Russia Says US Is Responsible for Deadly Ukrai...,"(1956, 1971)",[Russia Says US Is Responsible for Deadly Ukra...,14,The Kremlin said President Vladimir Putin had ...,"Dmitry Medvedev, Deputy Chairman of the Securi..."
412,EN,EN_UA_027787.txt,U.S. administration,2269,2287,Antagonist,[Terrorist],Russia Says US Is Responsible for Deadly Ukrai...,0,Russia Says US Is Responsible for Deadly Ukrai...,"(2254, 2273)",[Russia Says US Is Responsible for Deadly Ukra...,15,"Dmitry Medvedev, Deputy Chairman of the Securi...",He likened it to attacks carried out by gunmen...


In [13]:
# Example with a custom iterable
data = range(10, 101, 10)  # 10, 20, 30, ..., 100
data1 = range(10,101,10)
# Get the index of the greatest value
max_index = max(enumerate(data), key=lambda x: x[1])[0]

print(f"The index with the greatest value is: {max_index}")
print(data[max_index])
count = sum(1 for index in range(len(data)) if data[index] == data1[index])
print(count)

The index with the greatest value is: 9
100
10


In [14]:
# Example usage

texts = df_en['no_context']
subjects = df_en['entity']
labels = df_en['label']
candidate_labels = ["antagonist", "protagonist", "innocent"]
# Define the hypothesis template for entailment checking
hypothesis_template = "In this text {} is in the role of {}"

preds = []
for text,subject in zip(texts,subjects):
    hypothesis_template = "In this text {} is in the role of {}"

# Create the hypothesis for the span
    hypothesis_protagonist = hypothesis_template.format(subject, "protagonist")
    hypothesis_antagonist = hypothesis_template.format(subject, "antagonist")
    hypothesis_innocent = hypothesis_template.format(subject, "innocent")
    hypotheses = [hypothesis_protagonist, hypothesis_antagonist, hypothesis_innocent]
    result = classifier(text, hypotheses)
    res_txt = result['labels'][0]
    if 'protagonist' in res_txt:
        preds.append(1)
    elif 'antagonist' in res_txt:
        preds.append(0)
    elif 'innocent' in res_txt:
        preds.append(2)

    #print(result['labels'])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [15]:
print(len(labels),len(preds),len(texts),len(subjects))
s = labels.astype(int).tolist()
count = sum(1 for index in range(len(preds)) if preds[index] == s[index])
print(f"accuracy: {count/len(preds)}")

414 414 414 414
accuracy: 0.6352657004830918
